<a href="https://colab.research.google.com/github/Gongsoo/Python3/blob/main/titanic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/"

/content/drive/MyDrive


In [ ]:
!unzip -qq "/content/drive/MyDrive/타이타닉.zip"

In [ ]:
!mkdir titanic_pytorch


In [ ]:
import shutil
shutil.move('/content/drive/MyDrive/train.csv','/content/drive/MyDrive/titanic_pytorch/train.csv')
shutil.move('/content/drive/MyDrive/test.csv','/content/drive/MyDrive/titanic_pytorch/test.csv')
shutil.move('/content/drive/MyDrive/submission.csv','/content/drive/MyDrive/titanic_pytorch/submission.csv')

'/content/drive/MyDrive/titanic_pytorch/submission.csv'

In [ ]:
shutil.move('/content/drive/MyDrive/타이타닉.zip','/content/drive/MyDrive/titanic_pytorch/titanic_data.zip')

'/content/drive/MyDrive/titanic_pytorch/titanic_data.zip'

In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/train.csv')
test = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/submission.csv')

In [5]:
train = train.drop('Ticket',axis = 1)
train = train.drop('Cabin',axis = 1)
test = test.drop('Ticket',axis = 1)
test = test.drop('Cabin',axis = 1)

In [6]:
train = train.fillna({'Embarked':'S'})
test = test.fillna({'Embarked':'S','Fare':20})

In [7]:
em_map = {'S':1,'Q':2,'C':3}

for t in [train] :
  t['Embarked'] = t['Embarked'].map(em_map)
for t in [test] :
  t['Embarked'] = t['Embarked'].map(em_map)

In [8]:
s_map = {'male':1, 'female':2}

for t in [train] :
  t['Sex'] = t['Sex'].map(s_map)
for t in [test] :
  t['Sex'] = t['Sex'].map(s_map)

In [9]:
age_map = {'Mr.':20,'Miss.':15,'Mrs.':20,'Ms.':10,'Mme.':30,'Mlle.':15,'Dona.':30,'Countess.':30,'Sir.':25,'Lady.':20,'Jonkheer.':20,'Don.':20,'Col.':35,'Major.':30,'Capt.':23,'Dr.':30,'Master.':25,'Rev.':35}

for t in [train] :
  for i in range(len(t['Age'])) :
    if np.isnan(t['Age'][i]) :
      for j in age_map :
        if j in t['Name'][i] :
          t['Age'][i] = age_map[j]
for t in [test] :
  for i in range(len(t['Age'])) :
    if np.isnan(t['Age'][i]) :
      for j in age_map :
        if j in t['Name'][i] :
          t['Age'][i] = age_map[j]

<ipython-input-9-69fb9fd690b4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['Age'][i] = age_map[j]
<ipython-input-9-69fb9fd690b4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['Age'][i] = age_map[j]


In [10]:
train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)

In [11]:
train = train.drop(['PassengerId'], axis=1)
test = test.drop(['PassengerId'], axis=1)

In [28]:
x = train.drop(['Survived'],axis=1)
y = train['Survived']

In [13]:
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [29]:
scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

scaler.fit(test)
test_scaled = scaler.transform(test)

In [ ]:
print(type(y))

<class 'pandas.core.series.Series'>


In [30]:
train_x = torch.Tensor(x_scaled).float()
train_y = torch.Tensor(y).float()
test_x = torch.Tensor(test.values).float()

train_dataset = TensorDataset(train_x,train_y)
train_data_loader = DataLoader(train_dataset,batch_size = 64, shuffle = True,drop_last = True)


In [31]:
class LiModel(nn.Module) :
  def __init__(self):
    super(LiModel,self).__init__()
    self.layer1 = nn.Sequential(
        nn.Linear(7,1),
        nn.Sigmoid()
    )
    
  
  def forward(self, x) :
    x = self.layer1(x)
    return x
model = LiModel()
model

LiModel(
  (layer1): Sequential(
    (0): Linear(in_features=7, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [32]:

epochs = 500
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
train_loss_min = np.inf
for epoch in range(epochs) :
  train_loss = 0
  correct = 0
  for x,y in train_data_loader :
    
    model.train()
    optimizer.zero_grad()
    pred = model(x).view(-1)
    loss = torch.nn.functional.binary_cross_entropy(pred,y)

    
    loss.backward()
    optimizer.step()
    output = pred >= torch.FloatTensor([0.5])
    correct += output.float() == y
    train_loss += loss.item() * len(x)
  train_loss = train_loss / len(train_x)
  if (epoch + 1) % 20 == 0:
      print('Epoch {}/{}, Prediction : {}/{}, Cost : {}'.format(epoch+1, epochs, sum(correct), len(train_x), train_loss))
  if train_loss < train_loss_min:
      print('=*=*=*= Loss decreased ({:10f} ===> {:10f}). Saving the model! =*=*=*='.format(train_loss_min, train_loss))
      torch.save(model.state_dict(), 'model.pt')
      train_loss_min = train_loss

=*=*=*= Loss decreased (       inf ===>   0.571712). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.571712 ===>   0.520107). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.520107 ===>   0.482525). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.482525 ===>   0.459285). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.459285 ===>   0.441142). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.441142 ===>   0.430444). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.430444 ===>   0.421234). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.421234 ===>   0.413332). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.413332 ===>   0.411757). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.411757 ===>   0.405402). Saving the model! =*=*=*=
Epoch 20/500, Prediction : 660/891, Cost : 0.4210702829756988
Epoch 40/500, Prediction : 666/891, Cost : 0.41491275149429957
=*=*=*= Loss decreased (  0.405402 ===>   0.404113). Saving the model! =*=*=*=
Epoch 

In [34]:
model.eval()
with torch.no_grad():
  test_y_pred = model(test_x)

In [35]:
print(test_y_pred)

tensor([[1.9981e-08],
        [8.0742e-11],
        [7.7662e-14],
        [8.0421e-07],
        [3.9629e-05],
        [5.4769e-04],
        [6.6541e-07],
        [4.0343e-05],
        [2.7716e-04],
        [7.0904e-05],
        [2.3140e-05],
        [4.8179e-09],
        [7.8074e-01],
        [2.9810e-13],
        [1.1767e-06],
        [4.6338e-04],
        [7.3808e-08],
        [1.7002e-05],
        [1.8616e-06],
        [4.2722e-10],
        [6.2358e-09],
        [2.6089e-03],
        [1.5605e-02],
        [1.9087e-01],
        [1.0000e+00],
        [1.4539e-11],
        [3.6587e-01],
        [8.0841e-06],
        [1.0956e-07],
        [1.0842e-04],
        [1.8737e-10],
        [1.1386e-04],
        [5.3299e-07],
        [5.0588e-04],
        [1.1829e-03],
        [5.8735e-05],
        [1.0378e-03],
        [5.7967e-05],
        [2.4439e-06],
        [2.4163e-02],
        [5.2695e-09],
        [2.0555e-03],
        [6.9363e-10],
        [3.0874e-06],
        [9.2006e-07],
        [1

In [36]:
sub['Survived'] = torch.Tensor.detach(test_y_pred)
sub.to_csv('베이스 라인 .csv', index = False)